# Numpy

Python built-in collections like list offer a flexible way of storing and maniupulating data. As dicussed previously, collections usually just store references to objects. While this is every convenient when writing code, it comes with costs in performance in memory.

In [1]:
import random 
measurements = [random.randint(150, 200) for _ in range(1_000_000)]

def calculate_mean(measurements):
    accumulator = 0
    for measurement in measurements:
        accumulator += measurement
    
    mean = accumulator / len(measurements)
    return mean

%timeit calculate_mean(measurements)

24.9 ms ± 454 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


This is rather slow since Python has to rebind a new variable in every loop and then has to check whether the + operation is supported between the accumulator and the current measurement. This prevents it from trying to add together objects that can't be added, but in this case we are pretty sure that we are only dealing with integers. If we could tell the interpreter that we are only adding integers, we could skip all that typechecking and speed up the operation. For this purpose, numpy was invented.

In [2]:
measurements[:10]

[177, 153, 150, 184, 194, 163, 162, 163, 179, 167]

In [3]:
import numpy as np

In [4]:
measurements_array = np.array(measurements)
measurements_array

array([177, 153, 150, ..., 158, 191, 180])

In [5]:
type(measurements_array)

numpy.ndarray

They behave very similar to list, but have a fixed datatype underneath. Numpy automatically notices that all our values are intergers and chooses the appropriate datatype. An integer that takes up 64 bits of memory.

In [6]:
measurements_array[0]

177

In [7]:
measurements_array[0:5]

array([177, 153, 150, 184, 194])

In [8]:
measurements_array.dtype

dtype('int64')

In [9]:
measurements_array[13] = 0

In [10]:
measurements_array[14] = "cheesecake"

ValueError: invalid literal for int() with base 10: 'cheesecake'

Moreover, numpy offers a lot of routines for mathematical operations of arrays. Let's see if we acually gained something by using numpy.

In [ ]:
%timeit np.mean(measurements_array)

## Anatomy of arrays
Every array has a bunch of attributes that yield inforation about what it is.

### dtype

`.dtype` gives information about the data type. arrays can contain bools, ints, unsigned ints, floats or complex numbers of various byte sizes. They can also store strings or Python objects, but that has very few use cases.

In [ ]:
values = [0, 1, 2, 3, 4]
int_arr = np.array(values, dtype='int')
int_arr, int_arr.dtype

If the dtype does not match the given values, numpy will cast everything to that data type.

In [ ]:
bool_arr = np.array(values, dtype='bool')
bool_arr, bool_arr.dtype

If no explicit data type is given, numpy will choose the "smallest common denominator". In the following example, everything becomes a float, as ints can be represented as floats, but not vice versa.

In [ ]:
values = [0, 1, 2.5, 3, 4]
float_arr = np.array(values)
float_arr, float_arr.dtype

However, once the data type is set, everything will be coerced to that type.

In [ ]:
int_arr[1] = 2.5
int_arr, int_arr.dtype

These non-Python data types force us to again think about problems like overflow etc.

In [ ]:
values = [0, 1, 2, 3, 4]
uint_arr = np.array(values, dtype='int8')
uint_arr, uint_arr.dtype

In [ ]:
uint_arr[1] += 255
uint_arr

### shape and ndim
`.shape` is very important for keeping track of arrays with more than one dimension. It is a tuple with the number of elements in each dimension. `.ndim` is just the number of dimensions in total. 

In [ ]:
values = [0, 1, 2, 3, 4]
one_dim_arr = np.array(values)
one_dim_arr

In [ ]:
one_dim_arr.shape

In [ ]:
one_dim_arr.ndim

In [ ]:
values = [[0, 1, 2, 3, 4]] * 3
two_dim_arr = np.array(values)
two_dim_arr

In [ ]:
two_dim_arr.shape

In [ ]:
two_dim_arr.ndim

In [ ]:
values = [[[0, 1, 2, 3, 4]] * 3] * 6
three_dim_arr = np.array(values)
three_dim_arr

In [ ]:
three_dim_arr.shape

In [ ]:
three_dim_arr.ndim

## Creating arrays
We already saw how arrays can be created from Python lists (the same works with tuples). However, we often would like to create arrays directly, without creating Python objects. This can be accomplished by several utility functions.

The equivalent of `range`.

In [ ]:
np.arange(9)

In [ ]:
np.arange(start=2, stop=14, step=2)

Creating an array with a certain number of values in a certain interval.

In [ ]:
np.linspace(start=-5, stop=5, num=10)

An array containing zeros. The default `dtype` is `float`.

In [ ]:
np.zeros(5)

`np.zeros` takes a `shape` argument that lets us create multidimensional arrays.

In [ ]:
np.zeros(shape=(2, 3, 2))

The same goes for `ones`, `empty` and `full`.

In [ ]:
np.ones(shape=(2, 3, 2))

In [ ]:
# Corresponds to whatever was left in memory. Using zeros for initialising arrays is usually saver.
np.empty(shape=(2, 3, 2))

In [ ]:
np.full(shape=(2, 3, 2), fill_value=42)

`np.random` contains a lot of functions to create arrays filled with random values of various probability distributions.

In [ ]:
threebythree = np.random.random((3, 3))
threebythree

### Reshaping Arrays

In [ ]:
arr = np.arange(9)
arr

In [ ]:
arr = arr.reshape((9, 1)) #is now a column vector
arr, arr.shape

In [ ]:
arr = arr.reshape(threebythree.shape)
arr

In [ ]:
#transpose of that array
arr.T

In [ ]:
arr.T.reshape(arr.size)

## Mathematical operations
Numpy contains a lot of mathematical functions that operate on arrays in a vectorized manner. That means that they are applied to each element, without explicit for-loops. Vectorized functions are called `ufuncs` (universal functions) in Numpy.

### Standard arithmetic

In [ ]:
arr = np.arange(9)
arr

In [ ]:
arr * 3

In [ ]:
arr + arr

In [ ]:
arr - arr

In [ ]:
arr / arr

Numpy only warns if we're dividing by zero, and calculates the limit of that division (infinity/negative infinity) or, if for example calculating 0/0, it says the result is *not a number* (`nan`).

In [ ]:
a = np.arange(2)
a[0]/0, a[1]/0, np.log(a[0])

In [ ]:
arr * arr

In [ ]:
arr ** 2

Using `@` you can even do matrix multiplication. In the case of 1d arrays, this is the inner product between two vectors.

In [ ]:
arr @ arr

In [ ]:
# That's the same as
np.sum(arr * arr)

**For 2D-arrays:**

In [ ]:
arr = np.expand_dims(np.arange(9), axis=0) 
arr #is now a 2D-Row-Vector

In [ ]:
arr.T @ arr #Colum Vector * Row Vector

In [ ]:
arr @ arr.T #Row Vector * column Vector

In [ ]:
arr @ arr #Row Vector * Row Vector

### Some standard functions

In [ ]:
np.log(arr)

In [ ]:
np.exp(arr)

In [ ]:
np.sin(arr)

Always try to use vectorized ufuncs instead of explicit loops!

### Aggregations functions
Aggregation function are functions that reduce the dimensionality of an array. They provide an `axis` argument, to specify which dimension to reduce.

In [ ]:
np.random.seed(1)
two_dim_arr = np.random.randint(0, high=20, size=(4, 4))
two_dim_arr

If just the array is passed, the aggregation operation is performed over the whole array.

In [ ]:
np.min(two_dim_arr)

The optional `axis` argument allows us to specify, which dimension should be aggregated. You can think of it as the operation being applied to all entries that are obtained by keeping the indices in all dimensions fixed except for the `axis` dimension.
Let's look at the result of the minimum operation with `axis=0`:

In [ ]:
np.min(two_dim_arr, axis=0)

To illustrate what was just said, the entry at index `[1]`, i.e. `11` is the minimum of the following values:

In [ ]:
for i in range(4):
    print(two_dim_arr[i, 1])

Here we kept the second dimension fixed at `1` and only changed the first dimension aka. the dimension of index `0`.

The axis concept extends to more than one dimension

In [ ]:
np.random.seed(1)
three_dim_arr = np.random.randint(0, high=20, size=(4, 4, 4))
three_dim_arr

In [ ]:
np.min(three_dim_arr, axis=0)

Here the entry at index `[0, 0]`, i.e. `5` is the minimum of the following values. 

In [ ]:
for i in range(4):
    print(three_dim_arr[i, 0, 0])

Aggregation functions can also aggregate more than one dimension at once.

In [ ]:
np.min(three_dim_arr, axis=(1, 2))

Here the entry at index `[2]`, i.e. `3` is the minimum of the following values. 

In [ ]:
for i in range(4):
    for j in range(4):
        print(three_dim_arr[2, i, j])

Let's look at other aggregation functions.

In [ ]:
two_dim_arr

In [ ]:
np.max(two_dim_arr)

In [ ]:
np.max(two_dim_arr, axis=0)

In [ ]:
np.max(two_dim_arr, axis=1)

In [ ]:
np.sum(two_dim_arr)

In [ ]:
np.sum(two_dim_arr, axis=0)

In [ ]:
np.sum(two_dim_arr, axis=1)

Many of these function are also available as method on the array object.

In [ ]:
two_dim_arr.sum(axis=0)

## Advanced indexing
Numpy provides indexing methods that go beyond the indexing techniques known from standard Python sequences.


### Multidimensional indexing
Instead of doing subsequent indexing as with standard Python lists you can index all dimensions at once.

In [ ]:
two_dim_list = [
    [ 0,  1,  2],
    [ 3,  4,  5],
    [ 6,  7,  8],
    [ 9, 10, 11],
    [12, 13, 14]
]
two_dim_list[2][1]

In [ ]:
inner_list = two_dim_list[2]
inner_list[1]

In [ ]:
two_dim_arr = np.array(two_dim_list)
two_dim_arr[2, 1]

In [ ]:
two_dim_arr = np.array(two_dim_list)
two_dim_arr[2, 1]

You can use a colon to get all values from that dimensions.

In [ ]:
large_two_dim_arr = np.arange(81).reshape((9, 9))
large_two_dim_arr

In [ ]:
large_two_dim_arr[:, 1]

Standard slicing with `(start, stop, step)` works as expected.

In [ ]:
large_two_dim_arr[:, 1:3]

In [ ]:
large_two_dim_arr[:, 2:7:2]

Slices of an array are always `views`. That means, you just "view" the same chunk of meomory from a different perspective. This saves a lot of memory, but it means also that the original array will be changed, if you change the view.

In [ ]:
arr_slice = large_two_dim_arr[:, 1]
arr_slice[:] = 0
large_two_dim_arr

In [ ]:
large_two_dim_arr[:, 2] =0
large_two_dim_arr

### Fancy indexing
You can pass an array containing indices, this especially useful for drawing random items from an array.

In [ ]:
arr = np.arange(9) + 10
arr

In [ ]:
indices = np.array([1, 4, 5])
arr[indices]

The resulting array will reflect the shape of the index array.

In [ ]:
indices = np.array([[1, 4],
                    [5, 7]])
arr[indices]

You can index each dimension separately.

In [ ]:
import numpy as np

In [ ]:
two_dim_arr = np.array([
    [ 0,  1,  2],
    [ 3,  4,  5],
    [ 6,  7,  8],
    [ 9, 10, 11],
    [12, 13, 14]
    ])
two_dim_arr

In [ ]:
x_indices = np.array([3, 4])
y_indices = np.array([1, 2])
two_dim_arr[x_indices, y_indices] # Corresponds to indexing at [3, 1] and [4, 2].

### Masking
Logical arrays, i.e. arrays containing boolean values, can be used to index other arrays. These logical arrays are then called masks. This is especially useful to index based on logical conditions.

In [ ]:
# A simple integer array.
arr = np.arange(1, 6)
arr

In [ ]:
# A boolean array of the same shape as arr.
mask = np.array([True, False, True, False, True])
mask

Using the mask for indexing returns an array with only elements at positions where `mask` is `True`.  

In [ ]:
arr[mask]

The same can be used for assignment, which keeps the shape of the original array.

In [ ]:
arr[mask] = 10
arr

Mask can be created by using logical operators. For example, to get all the entries in an array that are greater than two.


In [ ]:
arr = np.arange(1, 6)
greater_two = arr > 2
greater_two


In [ ]:
arr[greater_two]

Or even shorter.

In [ ]:
arr[arr > 2]

Different masks can be combined using bitwise logical operators. These are the vectorized version of locial operators and should not be confounded with `and`, `or` and `not` with try to evaluated the truth value of a whole object.

In [ ]:
smaller_or_equal_four = arr <= 4
smaller_or_equal_four   

Bitwise and `&`.

In [ ]:
greater_two & smaller_or_equal_four

In [ ]:
# This does not work.
greater_two and smaller_or_equal_four

In [ ]:
arr[greater_two & smaller_or_equal_four]

Bitwise or using `|`.

In [ ]:
arr[greater_two | smaller_or_equal_four]

Bitwise xor using `^`.

In [ ]:
arr[greater_two ^ smaller_or_equal_four]

Bitwise negation using `~`.

In [ ]:
# Gives everything smaller or equal to 2.
arr[~greater_two]

### Combining arrays
There are many ways to combine existing arrays, like `np.append`, `np.concatenate` and `np.stack`. 

In [ ]:
np.concatenate((one_dim_arr, one_dim_arr))

In [ ]:
np.stack((one_dim_arr, one_dim_arr))

In [ ]:
np.append(one_dim_arr, one_dim_arr)

# More Python
Shameless self-promotion: The one-and-a-half-hour-version of this is available at Github, including a link to the video-version, done by Rüdiger Busche: https://github.com/scientificprogrammingUOS/lectures. Lecture 4 is about Numpy. As you'll also need Matplotlib, I also recommend lecture 5.